In [65]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gutot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [66]:
with open('treinamento.txt', 'r', encoding='utf-8') as f:
    treinamento = f.read()

treinamento[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

In [67]:
#Retirando caracteres especiais do treinamento
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras


In [68]:
lista_tokens = nltk.tokenize.word_tokenize(treinamento)
lista_palavras = separa_palavras(lista_tokens)
lista_palavras[:5]

['imagem', 'Temos', 'a', 'seguinte', 'classe']

In [69]:
#Colocando todo o treinamento em letras minúsculas
def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)
lista_normalizada[:5]

['imagem', 'temos', 'a', 'seguinte', 'classe']

In [70]:
# Veriicando o número de palavras únicas no treinamento
len(set(lista_normalizada))

18465

In [71]:
palavra_exemplo = 'programaão'

(palavra_exemplo[:8], palavra_exemplo[8:])

('programa', 'ão')

In [72]:
#Função para inserir letras no espaço faltante
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

insere_letras([('programa', 'ão')])[:5]

['programaaão', 'programabão', 'programacão', 'programadão', 'programaeão']

In [73]:
#Função para gerar palavras possíveis
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

palavras_geradas = gerador_palavras(palavra_exemplo)

palavras_geradas[:5]

['aprogramaão', 'bprogramaão', 'cprogramaão', 'dprogramaão', 'eprogramaão']

In [74]:
for palavra in palavras_geradas:
  if palavra == 'programação':
    print(f'Existe, a palavra correta é "{palavra}"')

Existe, a palavra correta é "programação"


In [75]:
#Foram geradas 517 palavras possíveis para encontrar a palavra "programação"
len(palavras_geradas)

517

In [76]:
#Para que o nosso corretor saiba qual a palavra correta, precisamos de um dicionário com as palavras e suas frequências
frequencia = nltk.FreqDist(lista_normalizada)

total_palavras = len(lista_normalizada)

frequencia.most_common(10)

[('de', 15502),
 ('o', 14056),
 ('que', 12230),
 ('a', 11099),
 ('e', 10501),
 ('para', 7710),
 ('um', 6368),
 ('é', 5899),
 ('uma', 5220),
 ('do', 5124)]

In [77]:
#Com o dicionário, podemos calcular a probabilidade de uma palavra aparecer no texto
def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada]/total_palavras


In [78]:
#Tendo a probabilidade, criamos a nossa função corretor
def corretor(palavra_errada):
    palavras_geradas = gerador_palavras(palavra_errada)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

Testes em palavras que faltam uma letra

In [79]:
palavra_exemplo = 'lgica'
corretor(palavra_exemplo)

'lógica'

In [80]:
palavra_exemplo = 'programaão'
corretor(palavra_exemplo)

'programação'

In [81]:
palavra_exemplo = 'pora'
corretor(palavra_exemplo)

'porta'

In [82]:
#Função para criar uma tupla com a palavra correta e a palavra errada
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, 'r', encoding='utf-8')
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

lista_teste = cria_dados_teste('palavras.txt')

In [83]:
def avaliador(testes):
    numero_palavras = len(testes)
    acertou = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras')

avaliador(lista_teste)

1.08% de 186 palavras


Uma letra a mais


In [84]:
#Função para deletar caracteres a mais de uma palavra
def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

exemplo = [('progr','samação')]

deletando_caracteres(exemplo)

['programação']

In [85]:
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    return palavras_geradas

palavra_exemplo = 'progrsamação'

palavras_geradas = gerador_palavras(palavra_exemplo)

print(palavras_geradas[:5])

['aprogrsamação', 'bprogrsamação', 'cprogrsamação', 'dprogrsamação', 'eprogrsamação']


In [86]:
for palavra in palavras_geradas:
  if palavra == 'programação':
    print(f'Existe, a palavra correta é "{palavra}"')

Existe, a palavra correta é "programação"


In [87]:
print(f'A palavra correta é {corretor(palavra_exemplo)}')
print(f'A palavra errada é {palavra_exemplo}')
print(f'Foram geradas {len(palavras_geradas)} palavras')

A palavra correta é programação
A palavra errada é progrsamação
Foram geradas 624 palavras


In [88]:
avaliador(lista_teste)

41.4% de 186 palavras


Trocando letras de lugar

In [89]:
#Função trocando caracteres
def troca_caracter(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

troca_caracter([('prog', 'tamação')])[:5]

['progaamação', 'progbamação', 'progcamação', 'progdamação', 'progeamação']

In [90]:
#Refatorando a função gerador de palavras
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_caracter(fatias)
    return palavras_geradas

palavra_exemplo = 'progtamação'

palavras_geradas = gerador_palavras(palavra_exemplo)

print(palavras_geradas[:5])

['aprogtamação', 'bprogtamação', 'cprogtamação', 'dprogtamação', 'eprogtamação']


In [91]:
for palavra in palavras_geradas:
  if palavra == 'programação':
    print(f'Existe, a palavra correta é "{palavra}"')

Existe, a palavra correta é "programação"


Invertendo letras

In [92]:
#Vamos juntar todas as nossas funções em uma só

######### Função para inserir letras no espaço faltante #########

def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

######### Função para deletar caracteres a mais de uma palavra #########

def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

######### Função trocando caracteres #########

def troca_caracter(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

######### Função para gerar palavras possíveis #########

def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra)+1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_caracter(fatias)
    return palavras_geradas


In [93]:
############ Avaliação do corretor ############

palavra_exemplo = 'prorgamação'

palavras_geradas = gerador_palavras(palavra_exemplo)

print(palavras_geradas[:5])


['aprorgamação', 'bprorgamação', 'cprorgamação', 'dprorgamação', 'eprorgamação']


In [94]:
for palavra in palavras_geradas:
  if palavra == 'programação':
    print(f'Existe, a palavra correta é "{palavra}"')



In [95]:
avaliador(lista_teste)

76.34% de 186 palavras


In [98]:
#Vamos tentar descobrir o que está causando o erro, sendo que pode ser tanto o corretor errando, quanto o fato de uma palavra não constar no arquivo de treinamento

def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
        else:
            desconhecida += (correta not in vocabulario)
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f'{taxa_acerto}% de {numero_palavras} palavras, desconhecidas é {taxa_desconhecida}%')

vocabulario = set(lista_normalizada)

avaliador(lista_teste, vocabulario)

76.34% de 186 palavras, desconhecidas é 6.99%


Preparando o corretor final


In [99]:
########################### Função insere_letras() ############################

# Recebe uma lista de tuplas (esquerdo, direito) que corresponde aos lados 
    # esquerdo e direito da palavra fatiada em dois
def insere_letras(fatias):

    # Criando uma lista vazia para armazenar as palavras corrigidas
    novas_palavras = []

    # Variável que armazena todas as letras do alfabeto e as vogais acentuadas
        # É daqui que nosso corretor pegará a letra faltante
    letras = 'abcedfghijklmnopqrstuvwxyzáâàãéêèíîìóôòõúûùç'

    # Iterando por todas as tuplas da lista recebida
    for esquerdo, direito in fatias:

        # Iterando por toda letra das variável letras
        for letra in letras:

            # Acrescentando todas as possibilidades de palavras possíveis
            novas_palavras.append(esquerdo + letra + direito)
    
    # Retornando uma lista de possíveis palavras
    return novas_palavras

######################## Função deletando_caracter() ##########################

# Função deletando_caracter()
# Recebe as fatias
def deletando_caracter(fatias):

    # Criando uma lista vazia para armazenar as palavras corrigidas
    novas_palavras = []

    # Iterando por todas as tuplas da lista recebida
    for esquerdo, direito in fatias:

        # Acrescentando todas as possibilidades de palavras possíveis
        novas_palavras.append(esquerdo + direito[1:])
    
    # Retornando uma lista de possíveis palavras
    return novas_palavras

######################## Função trocando_caracter() ###########################

# Função trocando_caracter()
# Recebe uma lista de tuplas (esquerdo, direito) que corresponde aos lados 
    # esquerdo e direito da palavra fatiada em dois
def troca_caracter(fatias):

    # Criando uma lista vazia para armazenar as palavras corrigidas
    novas_palavras = []

    # Variável que armazena todas as letras do alfabeto e as vogais acentuadas
        # É daqui que nosso corretor pegará a letra faltante
    letras = 'abcedfghijklmnopqrstuvwxyzáâàãéêèíîìóôòõúûùç'

    # Iterando por todas as tuplas da lista recebida
    for esquerdo, direito in fatias:

        # Iterando por toda letra das variável letras
        for letra in letras:

            # Acrescentando todas as possibilidades de palavras possíveis
            novas_palavras.append(esquerdo + letra + direito[1:])
    
    # Retornando uma lista de possíveis palavras
    return novas_palavras

####################### Função invertendo_caracter() ##########################

# Função invertendo_caracter()
# Recebe as fatias
def invertendo_caracter(fatias):

    # Criando uma lista vazia para armazenar as palavras corrigidas
    novas_palavras = []

    # Iterando por todas as tuplas da lista recebida
    for esquerdo, direito in fatias:
        
        # Selecionando apenas as fatias da direita que têm mais de uma letra, 
            # pois, se não, não há o que inverter
        if len(direito) > 1:
            
            # Acrescentando todas as possibilidades de palavras possíveis
            novas_palavras.append(esquerdo + direito[1] + direito[0] + direito[2:])
    
    # Retornando uma lista de possíveis palavras
    return novas_palavras

######################### Função gerador_palavras() ###########################

# Refatorando outra vez a função gerador_palavras()
def gerador_palavras(palavra):

    # Criando uma lista vazia para armazenar as duas fatias de cada palavra
    fatias = []

    # Iterando por cada letra de cada palavra
    for i in range(len(palavra) + 1):

        # Armazenando as duas fatias em uma tupla e essa tupla em uma lista
        fatias.append((palavra[:i], palavra[i:]))

    # Chamando a função insere_letras() com a lista de tuplas das fatias 
        # recém-criadas e armazenando o retorno dessa função em uma variável
    palavras_geradas = insere_letras(fatias)

    # Primeira refatoração
    palavras_geradas += deletando_caracter(fatias)

    # Segunda refatoração de fato ocorre
    palavras_geradas += troca_caracter(fatias)

    # Acrescentando outra função
    # É aqui que a terceira refatoração de fato ocorre
    palavras_geradas += invertendo_caracter(fatias)

    # Retornando a lista de possíveis palavras. A palavra correta estará aí no meio
    return palavras_geradas

############################# Função avaliador() ###############################

# Função avaliador()
# Recebe uma lista com as tuplas de palavras de teste para poder
    # avaliar o nosso corretor
def avaliador(testes, vocabulario):

    # Calculando o número de palavras da lista de teste
    numero_palavras = len(testes)

    # Setando os contadores
    acertou = desconhecidas = 0

    # Iterando por cada tupla dentro da lista de teste
    for correta, errada in testes:

        # Chamando a função corretor() passando cada palavra
            # digitada incorretamente
        palavra_corrigida = corretor(errada)

        # Incrementando o contador das palavras desconhecidas
        desconhecidas += (correta not in vocabulario)
        
        # Conferindo cada palavra para ver se ele conseguiu corrigir
        if palavra_corrigida == correta:

            # Incrementando o contador das palavras corretas
            acertou += 1
    
    # Calculando a taxa de acerto do nosso corretor
    taxa_acerto = round(acertou * 100 / numero_palavras, 2)

    # Calculando a taxa de erro referente às palavras desconhecidas
    taxa_desconhecidas = round(desconhecidas * 100 / numero_palavras, 2)

    # Mostrando a taxa de acerto doe nosso corretor
    print(f'{taxa_acerto}% de {numero_palavras} das palavras conhecidas\n'
          f'e {taxa_desconhecidas}% das palavras desconhecidas')

# Calculando as palavras conecidas
vocabulario = set(lista_normalizada)

# Chamando a função avaliador()
avaliador(lista_teste, vocabulario)

76.34% de 186 das palavras conhecidas
e 6.99% das palavras desconhecidas


In [110]:
teste = 'pessos'

corretor(teste)

'pessoas'

In [114]:
frase_teste = 'Todas as pessos desse lugar'

lista_teste = frase_teste.split()

lista_teste

lista_teste_corrigida = []

for palavra in lista_teste:
    lista_teste_corrigida.append(corretor(palavra))

lista_teste_corrigida

['todas', 'a', 'pessoas', 'esse', 'lugar']

In [115]:
from nltk.test.portuguese_en_fixt import setup_module

setup_module()

Skipped: portuguese_en.doctest imports nltk.examples.pt which doesn't exist!